# Speeding-up gradient-boosting

Neste notebook, apresentamos uma versão modificada do aumento de gradiente que usa um número reduzido de divisões ao construir as diferentes árvores. Esse algoritmo é chamado de "aumento de gradiente de histograma" no scikit-learn.

Mencionamos anteriormente que a floresta aleatória é um algoritmo eficiente, pois cada árvore do conjunto pode ser ajustada ao mesmo tempo de forma independente. Portanto, o algoritmo é escalonado de maneira eficiente com o número de núcleos e com o número de amostras.

No aumento de gradiente, o algoritmo é um algoritmo sequencial. É necessário que as `N-1` árvores tenham sido ajustadas para poder se ajustar à árvore no estágio `N`. Portanto, o algoritmo é bastante caro em termos computacionais. A parte mais cara desse algoritmo é a busca pela melhor divisão na árvore, que é uma abordagem de força bruta: todas as divisões possíveis são avaliadas e a melhor é escolhida. Explicamos este processo no caderno "árvore em profundidade", ao qual você pode se referir.

Para acelerar o algoritmo de aumento de gradiente, pode-se reduzir o número de divisões a serem avaliadas. Como consequência, o desempenho de generalização de tal árvore seria reduzido. No entanto, como estamos combinando várias árvores em um aumento de gradiente, podemos adicionar mais estimadores para superar esse problema.

Faremos uma implementação ingênua de tal algoritmo usando blocos de construção do scikit-learn. Primeiro, carregaremos o conjunto de dados de habitação da Califórnia.

In [1]:
from sklearn.datasets import fetch_california_housing

data, target = fetch_california_housing(return_X_y=True, as_frame=True)
target *= 100  # rescale the target in k$


<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last"> Se você quiser uma visão geral mais detalhada sobre este conjunto de dados, pode consultar o
Apêndice - seção de descrição dos conjuntos de dados no final deste MOOC. </p>
</div>

Faremos uma rápida avaliação do aumento de gradiente original.

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import GradientBoostingRegressor

gradient_boosting = GradientBoostingRegressor(n_estimators=200)
cv_results_gbdt = cross_validate(
    gradient_boosting, data, target, scoring="neg_mean_absolute_error",
    n_jobs=2
)

In [4]:
print("Gradient Boosting Decision Tree")
print(f"Mean absolute error via cross-validation: "
      f"{-cv_results_gbdt['test_score'].mean():.3f} +/- "
      f"{cv_results_gbdt['test_score'].std():.3f} k$")
print(f"Average fit time: "
      f"{cv_results_gbdt['fit_time'].mean():.3f} seconds")
print(f"Average score time: "
      f"{cv_results_gbdt['score_time'].mean():.3f} seconds")

Gradient Boosting Decision Tree
Mean absolute error via cross-validation: 46.389 +/- 2.910 k$
Average fit time: 10.796 seconds
Average score time: 0.022 seconds


Lembramos que uma forma de acelerar o aumento do gradiente é reduzir o número de divisões considerado dentro da construção da árvore. Uma maneira é agrupar os dados antes de colocá-los no aumento de gradiente. Um transformador chamado `KBinsDiscretizer` está fazendo essa transformação. Assim, podemos canalizar esse pré-processamento com o aumento de gradiente.

Podemos primeiro demonstrar a transformação feita pelo `KBinsDiscretizer`.

In [5]:
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(
    n_bins=256, encode="ordinal", strategy="quantile")
data_trans = discretizer.fit_transform(data)
data_trans

C:\Users\Rogerio Lopes\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
C:\Users\Rogerio Lopes\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 3 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
C:\Users\Rogerio Lopes\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 6 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
C:\Users\Rogerio Lopes\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:200: UserWarning: Bins whose width are too small (i.e., <= 

array([[249.,  39., 231., ...,  83., 162.,  30.],
       [248.,  19., 203., ...,  28., 161.,  30.],
       [242.,  49., 249., ..., 125., 160.,  29.],
       ...,
       [ 17.,  15., 126., ...,  49., 200.,  82.],
       [ 23.,  16., 136., ...,  29., 200.,  77.],
       [ 53.,  14., 130., ...,  93., 199.,  81.]])

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last"> A célula de código acima irá gerar alguns avisos. Na verdade, para alguns dos recursos, solicitamos muitos bins em relação à dispersão de dados para esses recursos. Os menores escaninhos serão removidos.

Vemos que o discretizador transforma os dados originais em um inteiro. Este inteiro representa o índice bin quando a distribuição por quantil é realizada. Podemos verificar o número de caixas por recurso. </p>
</div>

In [7]:
[len(np.unique(col)) for col in data_trans.T]

[256, 50, 256, 253, 256, 256, 207, 235]

Após essa transformação, vemos que temos no máximo 256 valores exclusivos por recursos. Agora, usaremos esse transformador para discretizar os dados antes de treinar o regressor de aumento de gradiente.

In [8]:
from sklearn.pipeline import make_pipeline

gradient_boosting = make_pipeline(
    discretizer, GradientBoostingRegressor(n_estimators=200))
cv_results_gbdt = cross_validate(
    gradient_boosting, data, target, scoring="neg_mean_absolute_error",
    n_jobs=2,
)

In [9]:
print("Gradient Boosting Decision Tree with KBinsDiscretizer")
print(f"Mean absolute error via cross-validation: "
      f"{-cv_results_gbdt['test_score'].mean():.3f} +/- "
      f"{cv_results_gbdt['test_score'].std():.3f} k$")
print(f"Average fit time: "
      f"{cv_results_gbdt['fit_time'].mean():.3f} seconds")
print(f"Average score time: "
      f"{cv_results_gbdt['score_time'].mean():.3f} seconds")

Gradient Boosting Decision Tree with KBinsDiscretizer
Mean absolute error via cross-validation: 45.806 +/- 2.194 k$
Average fit time: 9.336 seconds
Average score time: 0.036 seconds


Aqui, vemos que o tempo de ajuste foi drasticamente reduzido, mas que o desempenho de generalização do modelo é idêntico. O Scikit-learn oferece classes específicas que são ainda mais otimizadas para grandes conjuntos de dados, chamadas HistGradientBoostingClassifier e HistGradientBoostingRegressor. Cada recurso nos dados do conjunto de dados é primeiro categorizado por histogramas de computação, que são usados posteriormente para avaliar as possíveis divisões. O número de divisões a avaliar é muito menor. Esse algoritmo se torna muito mais eficiente do que o aumento de gradiente quando o conjunto de dados tem mais de 10.000 amostras.

A seguir, daremos um exemplo para um grande conjunto de dados e compararemos os tempos de computação com o experimento da seção anterior.

In [10]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

histogram_gradient_boosting = HistGradientBoostingRegressor(
    max_iter=200, random_state=0)
cv_results_hgbdt = cross_validate(
    histogram_gradient_boosting, data, target,
    scoring="neg_mean_absolute_error", n_jobs=2,
)

In [11]:
print("Histogram Gradient Boosting Decision Tree")
print(f"Mean absolute error via cross-validation: "
      f"{-cv_results_hgbdt['test_score'].mean():.3f} +/- "
      f"{cv_results_hgbdt['test_score'].std():.3f} k$")
print(f"Average fit time: "
      f"{cv_results_hgbdt['fit_time'].mean():.3f} seconds")
print(f"Average score time: "
      f"{cv_results_hgbdt['score_time'].mean():.3f} seconds")

Histogram Gradient Boosting Decision Tree
Mean absolute error via cross-validation: 43.758 +/- 2.694 k$
Average fit time: 3.142 seconds
Average score time: 0.050 seconds


O aumento de gradiente do histograma é o melhor algoritmo em termos de pontuação. Ele também será dimensionado quando o número de amostras aumentar, enquanto o reforço de gradiente normal não.